In [ ]:
%load_ext blackcellmagic

In [ ]:
import pandas as pd
import seaborn as sns
from IPython.display import display

import fig02 as src

signature_list = ["nps", "siips", "na-gen", "na-therm", "na-mech", "na-sound", "na-vis"]

cols = ["sub", "task", "group", "int", "unp"]
vas = (
    pd.concat(
        [
            pd.read_csv("../../results/cbf-ps-agg.csv").query(
                "task.str.endswith('49C')"
            )[cols],
            pd.read_csv("../../results/bold-ps-agg.csv")[cols],
        ]
    )
    .groupby("sub")
    .filter(lambda df: df["task"].nunique() == 3)  # filter missing data
    .melt(id_vars=["sub", "task", "group"], value_vars=["int", "unp"])
)

bold = (
    pd.read_csv("../../results/bold-ps-agg.csv")
    .query("FD < 0.5")  # also drops NaN
    .pipe(src.scale, signature_list)  # zscore
)

cbf = (
    pd.read_csv("../../results/cbf-ps-agg.csv")
    .query("task.str.endswith('49C')")  # select 49C runs
    .pipe(src.drop_outliers, "cbfQEI")  # -1.5 * IQR
    .pipe(src.scale, signature_list)  # zscore
)

brain_pattern_responses = (
    pd.concat([cbf.query("ses == 'mri2'"), bold])
    .groupby("sub", as_index=False)
    .apply(src.get_residual)
    .melt(
        id_vars=["sub", "task", "group", "int", "int_residual", "unp"],
        value_vars=signature_list,
    )
)


brain_pattern_responses.groupby("group")["sub"].nunique()


group x manipulation plots

In [ ]:
vas_plot = src.plot_group_by_manipulation(
    vas.query("group == 'c' or group == 'm'"),
    col_order=["int", "unp"],
    order=["baseline49C", "preManip49C", "postManip49C"],
    cmap=sns.color_palette(["#ABD0E6", "#3787C0"], n_colors=2)
)
vas_plot.set_ylabels("visual analog scale (0-10)")
vas_plot.savefig("../../figures/fig02-pain-ratings.png", dpi=300)

brain_pattern_responses_plot = src.plot_group_by_manipulation(
    brain_pattern_responses.query("group == 'c' or group == 'm'"),
    col_order=signature_list,
    cmap=sns.color_palette(["#ABD0E6", "#3787C0"], n_colors=2)
)
brain_pattern_responses_plot.set_ylabels("brain pattern reponses (AU)")
brain_pattern_responses_plot.savefig(
    "../../figures/fig02-brain-pattern-responses.png", dpi=300
)

mm_vs_placebo_plot = src.plot_group_by_manipulation(
    brain_pattern_responses.query(
        "sub.str.startswith('cbf') and (variable == 'nps' or variable == 'siips') and group != 'c'"
    ),
    col_order=["nps", "siips"],
    cmap=sns.color_palette(["#A39094", "#3787C0", "#D0AB8A"], n_colors=3)
)
mm_vs_placebo_plot.set_ylabels("brain pattern reponses (AU)")
mm_vs_placebo_plot.savefig("../../figures/fig02-mm-vs-placebo.png", dpi=300)

pain ratings vs brain pattern response

In [ ]:
corrwith_pain_m = (
    brain_pattern_responses.query("group == 'm'")
    .groupby("sub")
    .filter(lambda df: df["task"].nunique() == 2) # filter missing data
    .groupby(["sub", "variable"], as_index=False)
    .apply(src.get_corrwith_pain)
)

corrwith_pain_c = (
    brain_pattern_responses.query("group == 'c'")
    .groupby("sub")
    .filter(lambda df: df["task"].nunique() == 2) # filter missing data
    .groupby(["sub", "variable"], as_index=False)
    .apply(src.get_corrwith_pain)
)

m = src.plot_correlations(corrwith_pain_m, order=signature_list)
m.savefig("../../figures/fig02-corrwith-pain-m.png", dpi=300)

c = src.plot_correlations(corrwith_pain_c, order=signature_list)
c.savefig("../../figures/fig02-corrwith-pain-c.png", dpi=300)

a. mixed ANOVA: VAS pain ratings

In [ ]:
mixed_anova_df, pairwise_ttests_df = src.mixed_anova(
    vas.query("task != 'baseline49C' and (group == 'm' or group == 'c')"),
)
display(
    mixed_anova_df.style.background_gradient(cmap="Blues", subset=["np2"]),
    pairwise_ttests_df,
)

b. one sample ttest: correlation (r) with pain

In [ ]:
display(
    src.one_sample_ttest(corrwith_pain_m),
    src.one_sample_ttest(corrwith_pain_c),
)

c. mixed ANOVA: brain pattern responses

In [ ]:
mixed_anova_df, pairwise_ttests_df = src.mixed_anova(
    brain_pattern_responses.query("group == 'm' or group == 'c'")
)
display(
    mixed_anova_df.style.background_gradient(cmap="Blues", subset=["np2"]),
    pairwise_ttests_df,
)

d. mixed ANOVA: mindfulness vs placebo

In [ ]:
mixed_anova_df, pairwise_ttests_df = src.mixed_anova(
    brain_pattern_responses.query(
        "sub.str.startswith('cbf') and (variable == 'nps' or variable == 'siips') and group != 'c'"
    )
)
display(
    mixed_anova_df.style.background_gradient(cmap="Blues", subset=["np2"]),
    pairwise_ttests_df,
)